In [1]:
# various useful modules used in this report
import seaborn as sns
import pyspark.sql.functions as F
import pandas as pd
import matplotlib.pyplot as plt
from operator import add
from functools import reduce
import numpy as np
import re
%matplotlib inline

In [15]:
indi_full = spark.read.parquet("gs://voter-project-235-25/VM2Uniform--IN--2021-01-15_parq")

In [30]:
cols_to_keep = [
    "Residence_Families_HHCount",
    "Residence_HHGender_Description",
    "Mailing_Families_HHCount",
    "Mailing_HHGender_Description",
    "Parties_Description",
    "CommercialData_PropertyType",
    "Voters_VotingPerformanceEvenYearGeneral",
    "Voters_VotingPerformanceEvenYearPrimary",
    "Voters_VotingPerformanceEvenYearGeneralAndPrimary",
    "Voters_VotingPerformanceMinorElection",
    "AddressDistricts_Change_Changed_CD",
    "AddressDistricts_Change_Changed_SD",
    "AddressDistricts_Change_Changed_HD",
    "AddressDistricts_Change_Changed_County",
    "Residence_Addresses_Density",
    "Voters_Gender",
    "Voters_Age",
    "CommercialData_EstimatedHHIncome",
    "CommercialData_ISPSA",
    "CommercialData_AreaMedianEducationYears",
    "CommercialData_AreaMedianHousingValue",
    "CommercialData_MosaicZ4Global",
    "CommercialData_AreaPcntHHMarriedCoupleNoChild",
    "CommercialData_AreaPcntHHMarriedCoupleWithChild",
    "CommercialData_AreaPcntHHSpanishSpeaking",
    "CommercialData_AreaPcntHHWithChildren",
    "CommercialData_StateIncomeDecile",
    "Ethnic_Description",
    "EthnicGroups_EthnicGroup1Desc",
    "CommercialData_DwellingType",
    "CommercialData_PresenceOfChildrenCode",
    "CommercialData_PresenceOfPremCredCrdInHome",
    "CommercialData_DonatesToCharityInHome",
    "CommercialData_DwellingUnitSize",
    "CommercialData_ComputerOwnerInHome",
    "CommercialData_DonatesEnvironmentCauseInHome",
    "CommercialData_Education"
]


In [31]:
indi.printSchema()

root
 |-- Residence_Families_HHCount: integer (nullable = true)
 |-- Residence_HHGender_Description: string (nullable = true)
 |-- Mailing_Families_HHCount: integer (nullable = true)
 |-- Mailing_HHGender_Description: string (nullable = true)
 |-- Parties_Description: string (nullable = true)
 |-- CommercialData_PropertyType: string (nullable = true)
 |-- Voters_VotingPerformanceEvenYearGeneral: string (nullable = true)
 |-- Voters_VotingPerformanceEvenYearPrimary: string (nullable = true)
 |-- Voters_VotingPerformanceEvenYearGeneralAndPrimary: string (nullable = true)
 |-- Voters_VotingPerformanceMinorElection: string (nullable = true)
 |-- AddressDistricts_Change_Changed_CD: string (nullable = true)
 |-- AddressDistricts_Change_Changed_SD: string (nullable = true)
 |-- AddressDistricts_Change_Changed_HD: string (nullable = true)
 |-- AddressDistricts_Change_Changed_County: string (nullable = true)
 |-- Residence_Addresses_Density: integer (nullable = true)
 |-- Voters_Gender: string 

In [19]:
indi = (indi_full
        .select(cols_to_keep)
        .filter("Voters_Age > 39"))

In [20]:
indi.count()

2910248

In [27]:
# subset columns by regular expression
def select_matched(df, pattern, printSchema = False, void = False):
    """
    Function that selects the columns of `df` that match the regular expression `pattern`.
    Parameters:
        df -- PySpark dataframe
        pattern: string -- regular expression
        printSchema: bool -- If True, print schema of resulting dataframe
        void: bool -- If True, return None, otherwise, return resulting dataframe
    """
    pattern = re.compile(pattern)
    selected = [s for s in df.columns if pattern.match(s)]
    if (printSchema) & (not void):
        df = df.select(selected)
        df.printSchema()
        return df
    elif printSchema:
        df.select(selected).printSchema()
    elif not void:
        return df.select(selected)

In [34]:
select_matched(indi, "Voters").select("Voters_VotingPerformanceEvenYearGeneralAndPrimary").show()

+-------------------------------------------------+
|Voters_VotingPerformanceEvenYearGeneralAndPrimary|
+-------------------------------------------------+
|                                              64%|
|                                              50%|
|                                             100%|
|                                              71%|
|                                              78%|
|                                              64%|
|                                              71%|
|                                              92%|
|                                               0%|
|                                               0%|
|                                              78%|
|                                              71%|
|                                              92%|
|                                              92%|
|                                              21%|
|                                               7%|
|           

In [41]:
cs = select_matched(indi, ".*Performance.*").columns
cs

['Voters_VotingPerformanceEvenYearGeneral',
 'Voters_VotingPerformanceEvenYearPrimary',
 'Voters_VotingPerformanceEvenYearGeneralAndPrimary',
 'Voters_VotingPerformanceMinorElection']

In [43]:
indi1 = indi.alias('indi1')
for c in cs:
    indi1.withColumn(c, F.regexp_replace(F.col(c), "%", "").cast("integer"))


In [51]:
indi.select([F.col(c) for c in indi.columns if F.lit('%').isin(F.col(c))]).printSchema()

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.